In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True) 
import pandas as pd
import matplotlib.pyplot as plt
import csv

import random
random.seed(10)
np.random.seed(10)
features_path = ''
results_path =''
with open(results_path, mode='w') as result_file:
    results = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    raw_dataset_dataframe = pd.read_csv(features_path)
    is_nan = raw_dataset_dataframe.isnull()
    row_has_nan = is_nan.any(axis=1)
    rows_with_nan = raw_dataset_dataframe[row_has_nan]
    #prints
    X_df= raw_dataset_dataframe.filter(like='feature', axis=1)
    Y_df = raw_dataset_dataframe.filter(regex='label', axis=1)

    print(X_df.describe())
    correlation_matrix = X_df.corr()
    print(correlation_matrix)

    ################### Data partition ############################
    from sklearn.model_selection import train_test_split

    # train 70% and test 30%
    X_trainval, X_test, y_trainval, y_test = train_test_split(X_df, Y_df, test_size=0.3, random_state=10 ,stratify= Y_df)
    print('Training set size: {}'.format(X_trainval.shape)) 
    print('Test set size: {}'.format(X_test.shape)) 
    list_labels_train = list(y_trainval)
    list_labels_test = list(y_test)
    print('Number of samples healthy in train set: {}'.format(list_labels_train.count(0)))
    print('Number of samples osteoporosis in train set: {}'.format(list_labels_train.count(1)))
    print('Number of samples healthy in test set: {}'.format(list_labels_test.count(0)))
    print('Number of samples osteoporosis in test set: {}'.format(list_labels_test.count(1)))
        
    ######################  Data standardization   ########################

    mu = np.mean(X_trainval, axis=0)
    sigma = np.std(X_trainval, axis=0)
    X_trainval = (X_trainval - mu) / sigma
    X_test = (X_test - mu) / sigma

    ################## Training ##################
    from sklearn.model_selection import StratifiedKFold, cross_val_score
    cv = StratifiedKFold(n_splits=5, shuffle= True, random_state= 10)
        
    #---------------LOGISTIC REGRESSION-------------------#
        
    from sklearn.linear_model import LogisticRegression
    C_values = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
    regularizer = ['l1','l2','elasticnet']
    for reg in regularizer:
        regularizer = reg
        if regularizer == 'l2':
            best_solver = 'lbfgs'
        elif regularizer == 'l1': 
            best_solver = 'liblinear'
        elif regularizer == 'elasticnet': 
            best_solver = 'saga'
        else:
            raise ValueError('Unrecognized {} regularizer. This example only supports {}'.format(regularizer, ['l1', 'l2']))

          # inicializamos un vector donde guardaremos los valores de f1
        lr_f1_val = np.zeros(len(C_values))
        lr_reg_models = []
        lr_mean_f1 = np.zeros(len(C_values))
        lr_std_f1 = np.zeros(len(C_values))
        for i in range(len(C_values)):
            model = LogisticRegression(penalty=regularizer, fit_intercept=True, C=10**(C_values[i]), solver=best_solver, max_iter=15000)
            lr_reg_models.append(model)
            f1_per_fold = cross_val_score(model, X_trainval, y_trainval, 
                                                cv= cv, 
                                                scoring='f1')    
            lr_mean_f1[i] = np.mean(f1_per_fold)
            lr_std_f1[i] = np.std(f1_per_fold)
            results.writerow(['LR', regularizer, C_values[i], lr_mean_f1[i], lr_std_f1[i]])
            print('LR with C={} --> Acc={:.4f}+-{:.4f}'.format(C_values[i], lr_mean_f1[i], lr_std_f1[i]))
              i=i+1
        
        T_idx = np.argmax(lr_mean_f1)
        best_lr_performance = dict()
        best_lr_performance['mean']=lr_mean_f1[T_idx]
        best_lr_performance['std']=lr_std_f1[T_idx]
        print('-----------------------')
        print('Best C: {}'.format(C_values[T_idx]))
        print('Average f1: {:.4f}+-{:.4f}'.format(best_lr_performance['mean'], best_lr_performance['std']))
        print('-----------------------')


    #-----------------RANDOM FOREST------------------#
        
    from sklearn.ensemble import RandomForestClassifier
    num_trees = [10, 25, 50, 100, 200, 300]
    rf_models = []
    rf_mean_f1 = np.zeros(len(num_trees))
    rf_std_f1 = np.zeros(len(num_trees))
    i = 0
    for N in num_trees:
        model = RandomForestClassifier(n_estimators=N, random_state=10)
        rf_models.append(model)
        f1_per_fold = cross_val_score(model, X_trainval, y_trainval, 
                                            cv= cv, 
                                            scoring='f1')    
        rf_mean_f1[i] = np.mean(f1_per_fold)
        rf_std_f1[i] = np.std(f1_per_fold)
        results.writerow(['Random_Forest', 'NONE', num_trees[i], rf_mean_f1[i], rf_std_f1[i]])
        print('RF with num_trees={} --> Acc={:.4f}+-{:.4f}'.format(num_trees[i], rf_mean_f1[i], rf_std_f1[i]))
        i=i+1

   
    NT_idx = np.argmax(rf_mean_f1)
    best_rf_performance = dict()
    best_rf_performance['mean']=rf_mean_f1[NT_idx]
    best_rf_performance['std']=rf_std_f1[NT_idx]
    print('-----------------------')
    print('Best NT: {}'.format(num_trees[NT_idx]))
    print('Average f1: {:.4f}+-{:.4f}'.format(best_rf_performance['mean'], best_rf_performance['std']))
    print('-----------------------')

        
    #-----------------------Support Vector Machine-------------------------------#
    from sklearn.svm import SVC

    gamma_values = list(2.0**np.arange(-5, 5)) + ['auto', 'scale']
    C_values = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
    kernels = [ 'poly', 'rbf', 'sigmoid', 'linear'] 

    kernel_classifier_mean_f1 = np.zeros((len(C_values), len(gamma_values), len(kernels)))
    best_model = None
    current_f1 = -1.0
    for kernel in range(len(kernels)):
          for c_i in range(len(C_values)):
            for gamma_i in range(len(gamma_values)):
              # definimos el modelo
              model = SVC(probability = True, C=10**C_values[c_i], kernel=kernels[kernel], gamma=gamma_values[gamma_i])
              # usamos cross validation para evaluar la performance en cada fold
              f1_per_fold = cross_val_score(model, X_trainval, y_trainval, 
                                                cv= cv, 
                                                scoring='f1')
                if np.mean(f1_per_fold) > current_f1:
                    best_model = model
                    current_f1 = np.mean(f1_per_fold)
                kernel_classifier_mean_f1[c_i, gamma_i,kernel] = np.mean(f1_per_fold)
                results.writerow(['SVM', kernels[kernel], C_values[c_i], kernel_classifier_mean_f1[c_i, gamma_i,kernel], 0.0])
                print('Kernel SVM with KERNEL={} C={} gamma={} --> Acc={:.4f}'.format(kernels[kernel], 10**C_values[c_i], gamma_values[gamma_i], kernel_classifier_mean_f1[c_i, gamma_i,kernel]))
              

        with open('/home/calculo3/Documents/DOCTORADO/Dataset_TBS/csv/model_selection/svmnl_'+str(archivo)+'.pkl', 'wb') as f:
            pickle.dump({'model': best_model}, f)

        print (best_model)
        # buscamos el elemento con mejor f1
        indices = np.unravel_index(np.argmax(kernel_classifier_mean_f1), kernel_classifier_mean_f1.shape)
        print('-----------------------')
        print('Best C = {}'.format(10.0**C_values[indices[0]]))
        print('Best gamma = {}'.format(gamma_values[indices[1]]))
        print('Best kernel = {}'.format(kernels[indices[2]]))
        print('f1 on validation set = {:.4f}'.format(kernel_classifier_mean_f1[indices[0], indices[1],indices[2]]))
        print('-----------------------')   
        
    #-----------------------XGBoost-------------------------------#
    import xgboost as xgb
    from xgboost import XGBClassifier
    
    xgb_f1_val = 0
    xgb_models = []
    lrs = [0.001, 0.1, 0.3, 1.0]
    maxdepths = [4, 6, 8, 10]
    n_es = [1, 10, 50, 100, 500]
    current_f1=0
    xgb_mean_f1 = np.zeros((len(lrs), len(maxdepths), len(n_es)))
    xgb_std_f1 = np.zeros((len(lrs), len(maxdepths), len(n_es)))
    i = 0 
    for lr in range(len(lrs)):
        for maxdepth in range(len(maxdepths)):
            for ne in range(len(n_es)):        
                model = xgb.XGBClassifier(n_estimators= n_es[ne], max_depth = maxdepths[maxdepth], learning_rate = lrs[lr], objective='binary:logistic', use_label_encoder=False,random_state=42)
                xgb_models.append(model)
                f1_per_fold = cross_val_score(model, X_trainval, y_trainval, cv= cv, scoring='f1')   
                if np.mean(accuracy_per_fold) > current_accuracy:
                    best_model = model
                    current_f1 = np.mean(f1_per_fold)
                    xgb_mean_f1[lr, maxdepth, ne] = np.mean(f1_per_fold)
                    xgb_std_f1[lr, maxdepth, ne] = np.std(f1_per_fold)
                    # imprimimos
                    results.writerow(['xgboost', n_es[ne], maxdepths[maxdepth], lrs[lr], xgb_mean_f1[lr, maxdepth, ne], xgb_std_f1[lr, maxdepth, ne]])
                    if verbose:
                        print('XGB with ne={} maxdepth={} lr={} --> Acc={:.4f}+-{:.4f}'.format(n_es[ne], maxdepths[maxdepth], lrs[lr],xgb_mean_f1[lr, maxdepth, ne], xgb_std_f1[lr, maxdepth, ne]))          

Training set size: (105, 3)
Test set size: (46, 3)
Cantidad de casos de Control en entrenamiento/val: 62
Cantidad de casos de Osteoporosis en entrenamiento/val: 43
Cantidad de casos de Control en test: 27
Cantidad de casos de Osteoporosis en test: 19
REG LOG with C=-3 --> Acc=0.5905+-0.0233
REG LOG with C=-2 --> Acc=0.5905+-0.0233
REG LOG with C=-1 --> Acc=0.6571+-0.1256
REG LOG with C=0 --> Acc=0.7143+-0.1413
REG LOG with C=1 --> Acc=0.6857+-0.1299
REG LOG with C=2 --> Acc=0.6857+-0.1299
REG LOG with C=3 --> Acc=0.6857+-0.1299
-----------------------
Best C: 0
Average accuracy: 0.7143+-0.1413
-----------------------
REG LOG with C=-3 --> Acc=0.5905+-0.0233
REG LOG with C=-2 --> Acc=0.6000+-0.0381
REG LOG with C=-1 --> Acc=0.6857+-0.0883
REG LOG with C=0 --> Acc=0.6952+-0.1263
REG LOG with C=1 --> Acc=0.6952+-0.1299
REG LOG with C=2 --> Acc=0.6857+-0.1299
REG LOG with C=3 --> Acc=0.6857+-0.1299
-----------------------
Best C: 0
Average accuracy: 0.6952+-0.1263
-----------------------
S

/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3

SVM with C=2 --> Acc=0.6381+-0.1069
SVM with C=3 --> Acc=0.6381+-0.1069
-----------------------
Best C: 0
Average accuracy: 0.6952+-0.1069
-----------------------
RF with num_trees=10 --> Acc=0.6667+-0.0301
RF with num_trees=25 --> Acc=0.6667+-0.0602
RF with num_trees=50 --> Acc=0.6476+-0.0486
RF with num_trees=100 --> Acc=0.6667+-0.0522
RF with num_trees=200 --> Acc=0.6476+-0.0646
RF with num_trees=300 --> Acc=0.6286+-0.0819
-----------------------
Best NT: 10
Average accuracy: 0.6667+-0.0301
-----------------------
KNN with K_value=1 --> Acc=0.7333+-0.0646
KNN with K_value=3 --> Acc=0.7429+-0.0233
KNN with K_value=5 --> Acc=0.7238+-0.0700
KNN with K_value=7 --> Acc=0.6857+-0.0646
KNN with K_value=9 --> Acc=0.7048+-0.0555
KNN with K_value=15 --> Acc=0.6667+-0.0797
KNN with K_value=25 --> Acc=0.6857+-0.0571
-----------------------
Best K: 3
Average accuracy: 0.7429+-0.0233
-----------------------
SVM with RBF kernel
Kernel SVM with KERNEL=poly C=0.001 gamma=0.03125 --> Acc=0.5905
Kerne

Kernel SVM with KERNEL=rbf C=1 gamma=4.0 --> Acc=0.7238
Kernel SVM with KERNEL=rbf C=1 gamma=8.0 --> Acc=0.7143
Kernel SVM with KERNEL=rbf C=1 gamma=16.0 --> Acc=0.7238
Kernel SVM with KERNEL=rbf C=1 gamma=auto --> Acc=0.6857
Kernel SVM with KERNEL=rbf C=1 gamma=scale --> Acc=0.6762
Kernel SVM with KERNEL=rbf C=10 gamma=0.03125 --> Acc=0.6762
Kernel SVM with KERNEL=rbf C=10 gamma=0.0625 --> Acc=0.6762
Kernel SVM with KERNEL=rbf C=10 gamma=0.125 --> Acc=0.6762
Kernel SVM with KERNEL=rbf C=10 gamma=0.25 --> Acc=0.7048
Kernel SVM with KERNEL=rbf C=10 gamma=0.5 --> Acc=0.7429
Kernel SVM with KERNEL=rbf C=10 gamma=1.0 --> Acc=0.7333
Kernel SVM with KERNEL=rbf C=10 gamma=2.0 --> Acc=0.7143
Kernel SVM with KERNEL=rbf C=10 gamma=4.0 --> Acc=0.6857
Kernel SVM with KERNEL=rbf C=10 gamma=8.0 --> Acc=0.6762
Kernel SVM with KERNEL=rbf C=10 gamma=16.0 --> Acc=0.6857
Kernel SVM with KERNEL=rbf C=10 gamma=auto --> Acc=0.7333
Kernel SVM with KERNEL=rbf C=10 gamma=scale --> Acc=0.7143
Kernel SVM with KE

/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3

...

Kernel SVM with KERNEL=rbf C=1 gamma=16.0 --> Acc=0.5905
Kernel SVM with KERNEL=rbf C=1 gamma=auto --> Acc=0.7238
Kernel SVM with KERNEL=rbf C=1 gamma=scale --> Acc=0.7238
Kernel SVM with KERNEL=rbf C=10 gamma=0.03125 --> Acc=0.7048
Kernel SVM with KERNEL=rbf C=10 gamma=0.0625 --> Acc=0.7143
Kernel SVM with KERNEL=rbf C=10 gamma=0.125 --> Acc=0.7143
Kernel SVM with KERNEL=rbf C=10 gamma=0.25 --> Acc=0.6952
Kernel SVM with KERNEL=rbf C=10 gamma=0.5 --> Acc=0.6857
Kernel SVM with KERNEL=rbf C=10 gamma=1.0 --> Acc=0.6381
Kernel SVM with KERNEL=rbf C=10 gamma=2.0 --> Acc=0.6000
Kernel SVM with KERNEL=rbf C=10 gamma=4.0 --> Acc=0.5905
Kernel SVM with KERNEL=rbf C=10 gamma=8.0 --> Acc=0.5905
Kernel SVM with KERNEL=rbf C=10 gamma=16.0 --> Acc=0.5905
Kernel SVM with KERNEL=rbf C=10 gamma=auto --> Acc=0.7143
Kernel SVM with KERNEL=rbf C=10 gamma=scale --> Acc=0.7143
Kernel SVM with KERNEL=rbf C=100 gamma=0.03125 --> Acc=0.6952
Kernel SVM with KERNEL=rbf C=100 gamma=0.0625 --> Acc=0.7048
Kernel 

REG LOG with C=2 --> Acc=0.6952+-0.0486
REG LOG with C=3 --> Acc=0.6857+-0.0381
-----------------------
Best C: 1
Average accuracy: 0.7143+-0.0673
-----------------------
REG LOG with C=-3 --> Acc=0.5905+-0.0233
REG LOG with C=-2 --> Acc=0.6381+-0.0774
REG LOG with C=-1 --> Acc=0.7048+-0.0971
REG LOG with C=0 --> Acc=0.7333+-0.0981
REG LOG with C=1 --> Acc=0.7524+-0.1061
REG LOG with C=2 --> Acc=0.7048+-0.0467
REG LOG with C=3 --> Acc=0.6857+-0.0486
-----------------------
Best C: 1
Average accuracy: 0.7524+-0.1061
-----------------------
SVM with C=-3 --> Acc=0.6762+-0.1017
SVM with C=-2 --> Acc=0.7048+-0.0873
SVM with C=-1 --> Acc=0.7429+-0.0981
SVM with C=0 --> Acc=0.7524+-0.1061
SVM with C=1 --> Acc=0.7333+-0.0774
SVM with C=2 --> Acc=0.7143+-0.0426


/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/calculo3/anaconda3/envs/gpu/lib/python3

SVM with C=3 --> Acc=0.6571+-0.0190
-----------------------
Best C: 0
Average accuracy: 0.7524+-0.1061
-----------------------
RF with num_trees=10 --> Acc=0.7048+-0.1256
RF with num_trees=25 --> Acc=0.6762+-0.1143
RF with num_trees=50 --> Acc=0.6952+-0.1026
RF with num_trees=100 --> Acc=0.7143+-0.0852
RF with num_trees=200 --> Acc=0.7333+-0.0933
RF with num_trees=300 --> Acc=0.7333+-0.0933
-----------------------
Best NT: 200
Average accuracy: 0.7333+-0.0933
-----------------------
KNN with K_value=1 --> Acc=0.7619+-0.0301
KNN with K_value=3 --> Acc=0.7333+-0.0486
KNN with K_value=5 --> Acc=0.6667+-0.0797
KNN with K_value=7 --> Acc=0.6571+-0.0356
KNN with K_value=9 --> Acc=0.6762+-0.0873
KNN with K_value=15 --> Acc=0.6571+-0.0700
KNN with K_value=25 --> Acc=0.6571+-0.0923
-----------------------
Best K: 1
Average accuracy: 0.7619+-0.0301
-----------------------
SVM with RBF kernel
Kernel SVM with KERNEL=poly C=0.001 gamma=0.03125 --> Acc=0.5905
Kernel SVM with KERNEL=poly C=0.001 gamm

Kernel SVM with KERNEL=rbf C=1 gamma=0.5 --> Acc=0.6952
Kernel SVM with KERNEL=rbf C=1 gamma=1.0 --> Acc=0.6476
Kernel SVM with KERNEL=rbf C=1 gamma=2.0 --> Acc=0.6286
Kernel SVM with KERNEL=rbf C=1 gamma=4.0 --> Acc=0.6095
Kernel SVM with KERNEL=rbf C=1 gamma=8.0 --> Acc=0.5905
Kernel SVM with KERNEL=rbf C=1 gamma=16.0 --> Acc=0.5905
Kernel SVM with KERNEL=rbf C=1 gamma=auto --> Acc=0.7048
Kernel SVM with KERNEL=rbf C=1 gamma=scale --> Acc=0.7048
Kernel SVM with KERNEL=rbf C=10 gamma=0.03125 --> Acc=0.7429
Kernel SVM with KERNEL=rbf C=10 gamma=0.0625 --> Acc=0.6857
Kernel SVM with KERNEL=rbf C=10 gamma=0.125 --> Acc=0.7238
Kernel SVM with KERNEL=rbf C=10 gamma=0.25 --> Acc=0.7143
Kernel SVM with KERNEL=rbf C=10 gamma=0.5 --> Acc=0.7048
Kernel SVM with KERNEL=rbf C=10 gamma=1.0 --> Acc=0.6857
Kernel SVM with KERNEL=rbf C=10 gamma=2.0 --> Acc=0.6190
Kernel SVM with KERNEL=rbf C=10 gamma=4.0 --> Acc=0.6190
Kernel SVM with KERNEL=rbf C=10 gamma=8.0 --> Acc=0.5905
Kernel SVM with KERNEL=rb